In [ ]:
import os

import httpx
from dotenv import load_dotenv

load_dotenv()

In [ ]:
__username = os.getenv("API_ACCESS_KEY")
__password = os.getenv("API_SECRET_KEY")

__headers = {
    "x-chain-id": "8217",
    "Content-Type": "application/json",
}

In [ ]:
__address = "0x930c791D16702e8116ff64DA06C07Ee946F84Aa6"
__url = f"https://th-api.klaytnapi.com/v2/account/{__address}/token"


__params = {
    "kind": "nft",
}

# 4.8s
__resp = httpx.get(
    __url, headers=__headers, params=__params, auth=(__username, __password)
)

__resp = __resp.json()

for __item in __resp["items"]:
    if "extras" in __item:
        if "tokenUri" in __item["extras"]:
            __item["extras"]["tokenUri"] = httpx.get(
                __item["extras"]["tokenUri"]
            ).json()

__resp

In [ ]:
len(__resp["items"])

In [ ]:
%pip install aiohttp

In [ ]:
import asyncio

import aiohttp


async def fetch(session, url):
    async with session.get(url) as response:
        return await response.json()

async def fetch_all(urls, loop):
    async with aiohttp.ClientSession(loop=loop) as session:
        results = await asyncio.gather(*[fetch(session, url) for url in urls], return_exceptions=True)
        return results

async def main():
    async with aiohttp.ClientSession() as session:
        resp = await session.get(__url, headers=__headers, params=__params, auth=aiohttp.BasicAuth(__username, __password))
        resp = await resp.json()

        token_uris = [item["extras"]["tokenUri"] for item in resp["items"] if "extras" in item and "tokenUri" in item["extras"]]

        results = await fetch_all(token_uris, asyncio.get_event_loop())

        for item, result in zip(resp["items"], results):
            if "extras" in item and "tokenUri" in item["extras"]:
                item["extras"]["tokenUri"] = result

        print(resp)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())